This example notebook closely follows the [Databricks documentation](https://docs.azuredatabricks.net/spark/latest/data-sources/azure/azure-datalake.html) for how to set up Azure Data Lake Store as a data source in Databricks.

### 0 - Setup

To get set up, do these tasks first: 

- Get service credentials: Client ID `<aaaaaaaa-bbbb-cccc-dddd-eeeeeeeeeeee>` and Client Credential `<NzQzY2QzYTAtM2I3Zi00NzFmLWI3MGMtMzc4MzRjZmk=>`. Follow the instructions in [Create service principal with portal](https://docs.microsoft.com/en-us/azure/azure-resource-manager/resource-group-create-service-principal-portal). 
- Get directory ID `<ffffffff-gggg-hhhh-iiii-jjjjjjjjjjjj>`: This is also referred to as *tenant ID*. Follow the instructions in [Get tenant ID](https://docs.microsoft.com/en-us/azure/azure-resource-manager/resource-group-create-service-principal-portal#get-tenant-id). 
- If you haven't set up the service app, follow this [tutorial](https://docs.microsoft.com/en-us/azure/azure-databricks/databricks-extract-load-sql-data-warehouse). Set access at the root directory or desired folder level to the service or everyone.

There are two options to read and write Azure Data Lake data from Azure Databricks:
1. DBFS mount points
2. Spark configs

## 1 - DBFS mount points
[DBFS](https://docs.azuredatabricks.net/user-guide/dbfs-databricks-file-system.html) mount points let you mount Azure Data Lake Store for all users in the workspace. Once it is mounted, the data can be accessed directly via a DBFS path from all clusters, without the need for providing credentials every time. The example below shows how to set up a mount point for Azure Data Lake Store.

In [0]:
dbutils.fs.unmount('/mnt/lake')

/mnt/lake has been unmounted.
Out[2]: True

In [0]:
dbutils.fs.mount(
  source = "wasbs://container1@projectaccount123.blob.core.windows.net",
  mount_point = '/mnt/lake',
  extra_configs = {
    'fs.azure.account.key.projectaccount123.blob.core.windows.net':'c9F/bns3QWX59Hx7IDic1V9iSaoB17Gk8y5f0npreZMROsyQ6ausbUeBBUmpGjdtKq17pnBwSOMq+AStiJsEnQ=='
  }
  
)

Out[3]: True

In [0]:
dbutils.fs.ls('/mnt/lake')

Out[4]: [FileInfo(path='dbfs:/mnt/lake/superstore.csv', name='superstore.csv', size=1000478, modificationTime=1667300884000)]

In [0]:
!pip install pyspark

Collecting pyspark
 Downloading pyspark-3.3.1.tar.gz (281.4 MB)
 | 10 kB 23.2 MB/s eta 0:00:13 | | 20 kB 29.3 MB/s eta 0:00:10 | | 30 kB 22.7 MB/s eta 0:00:13 | | 40 kB 17.8 MB/s eta 0:00:16 | | 51 kB 20.2 MB/s eta 0:00:14 | | 61 kB 22.7 MB/s eta 0:00:13 | | 71 kB 19.2 MB/s eta 0:00:15 | | 81 kB 15.4 MB/s eta 0:00:19 | | 92 kB 16.7 MB/s eta 0:00:17 | | 102 kB 17.9 MB/s eta 0:00:16 | | 112 kB 17.9 MB/s eta 0:00:16 | | 122 kB 17.9 MB/s eta 0:00:16 | | 133 kB 17.9 MB/s eta 0:00:16 | | 143 kB 17.9 MB/s eta 0:00:16 | | 153 kB 17.9 MB/s eta 0:00:16 | | 163 kB 17.9 MB/s eta 0:00:16 | | 174 kB 17.9 MB/s eta 0:00:16 | | 184 kB 17.9 MB/s eta 0:00:16 | | 194 kB 17.9 MB/s eta 0:00:16 | | 204 kB 17.9 MB/s eta 0:00:16 | | 215 kB 17.9 MB/s eta 0:00:16 | | 225 kB 17.9 MB/s eta 0:00:16 | | 235 kB 17.9 MB/s eta 0:00:16 | | 245 kB 17.9 MB/s eta 0:00:16 | | 256 kB 17.9 MB/s eta 0:00:16 | | 266 kB 17.9 MB/s eta 0:00:16 | | 276 kB 17.9 MB/s eta 0:00:16 | | 286 kB 17.9 MB/s eta 0:00:16 | | 296 kB 17.9 MB/s eta 0:00:16 | | 307 kB 17.9 MB/s eta 0:00:16 | | 317 kB 17.9 MB/s eta 0:00:16 | | 327 kB 17.9 MB/s eta 0:00:16 | | 337 kB 17.9 MB/s eta 0:00:16 | | 348 kB 17.9 MB/s eta 0:00:16 | | 358 kB 17.9 MB/s eta 0:00:16 | | 368 kB 17.9 MB/s eta 0:00:16 | | 378 kB 17.9 MB/s eta 0:00:16 | | 389 kB 17.9 MB/s eta 0:00:16 | | 399 kB 17.9 MB/s eta 0:00:16 | | 409 kB 17.9 MB/s eta 0:00:16 | | 419 kB 17.9 MB/s eta 0:00:16 | | 430 kB 17.9 MB/s eta 0:00:16 | | 440 kB 17.9 MB/s eta 0:00:16 | | 450 kB 17.9 MB/s eta 0:00:16 | | 460 kB 17.9 MB/s eta 0:00:16 | | 471 kB 17.9 MB/s eta 0:00:16 | | 481 kB 17.9 MB/s eta 0:00:16 | | 491 kB 17.9 MB/s eta 0:00:16 | | 501 kB 17.9 MB/s eta 0:00:16 | | 512 kB 17.9 MB/s eta 0:00:16 | | 522 kB 17.9 MB/s eta 0:00:16 | | 532 kB 17.9 MB/s eta 0:00:16 | | 542 kB 17.9 MB/s eta 0:00:16 | | 552 kB 17.9 MB/s eta 0:00:16 | | 563 kB 17.9 MB/s eta 0:00:16 | | 573 kB 17.9 MB/s eta 0:00:16 | | 583 kB 17.9 MB/s eta 0:00:16 | | 593 kB 17.9 MB/s eta 0:00:16 | | 604 kB 17.9 MB/s eta 0:00:16 | | 614 kB 17.9 MB/s eta 0:00:16 | | 624 kB 17.9 MB/s eta 0:00:16 | | 634 kB 17.9 MB/s eta 0:00:16 | | 645 kB 17.9 MB/s eta 0:00:16 | | 655 kB 17.9 MB/s eta 0:00:16 | | 665 kB 17.9 MB/s eta 0:00:16 | | 675 kB 17.9 MB/s eta 0:00:16 | | 686 kB 17.9 MB/s eta 0:00:16 | | 696 kB 17.9 MB/s eta 0:00:16 | | 706 kB 17.9 MB/s eta 0:00:16 | | 716 kB 17.9 MB/s eta 0:00:16 | | 727 kB 17.9 MB/s eta 0:00:16 | | 737 kB 17.9 MB/s eta 0:00:16 | | 747 kB 17.9 MB/s eta 0:00:16 | | 757 kB 17.9 MB/s eta 0:00:16 | | 768 kB 17.9 MB/s eta 0:00:16 | | 778 kB 17.9 MB/s eta 0:00:16 | | 788 kB 17.9 MB/s eta 0:00:16 | | 798 kB 17.9 MB/s eta 0:00:16 | | 808 kB 17.9 MB/s eta 0:00:16 | | 819 kB 17.9 MB/s eta 0:00:16 | | 829 kB 17.9 MB/s eta 0:00:16 | | 839 kB 17.9 MB/s eta 0:00:16 | | 849 kB 17.9 MB/s eta 0:00:16 | | 860 kB 17.9 MB/s eta 0:00:16 | | 870 kB 17.9 MB/s eta 0:00:16 | | 880 kB 17.9 MB/s eta 0:00:16 | | 890 kB 17.9 MB/s eta 0:00:16 | | 901 kB 17.9 MB/s eta 0:00:16 | | 911 kB 17.9 MB/s eta 0:00:16 | | 921 kB 17.9 MB/s eta 0:00:16 | | 931 kB 17.9 MB/s eta 0:00:16 | | 942 kB 17.9 MB/s eta 0:00:16 | | 952 kB 17.9 MB/s eta 0:00:16 | | 962 kB 17.9 MB/s eta 0:00:16 | | 972 kB 17.9 MB/s eta 0:00:16 |▏ | 983 kB 17.9 MB/s eta 0:00:16 |▏ | 993 kB 17.9 MB/s eta 0:00:16 |▏ | 1.0 MB 17.9 MB/s eta 0:00:16 |▏ | 1.0 MB 17.9 MB/s eta 0:00:16 |▏ | 1.0 MB 17.9 MB/s eta 0:00:16 |▏ | 1.0 MB 17.9 MB/s eta 0:00:16 |▏ | 1.0 MB 17.9 MB/s eta 0:00:16 |▏ | 1.1 MB 17.9 MB/s eta 0:00:16 |▏ | 1.1 MB 17.9 MB/s eta 0:00:16 |▏ | 1.1 MB 17.9 MB/s eta 0:00:16 |▏ | 1.1 MB 17.9 MB/s eta 0:00:16 |▏ | 1.1 MB 17.9 MB/s eta 0:00:16 |▏ | 1.1 MB 17.9 MB/s eta 0:00:16 |▏ | 1.1 MB 17.9 MB/s eta 0:00:16 |▏ | 1.1 MB 17.9 MB/s eta 0:00:16 |▏ | 1.1 MB 17.9 MB/s eta 0:00:16 |▏ | 1.1 MB 17.9 MB/s eta 0:00:16 |▏ | 1.2 MB 17.9 MB/s eta 0:00:16 |▏ | 1.2 MB 17.9 MB/s eta 0:00:16 |▏ | 1.2 MB 17.9 MB/s eta 0:00:16 |▏ | 1.2 MB 17

In [0]:
import pandas as pd
df = pd.read_csv('/dbfs/mnt/lake/superstore.csv')

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 9 columns):
 # Column Non-Null Count Dtype 
--- ------ -------------- ----- 
 0 Order ID 9994 non-null object 
 1 Ship Mode 9994 non-null object 
 2 Segment 9994 non-null object 
 3 Region 9994 non-null object 
 4 Product ID 9994 non-null object 
 5 Sales 9994 non-null float64
 6 Quantity 9994 non-null int64 
 7 Discount 9994 non-null object 
 8 Profit 9994 non-null float64
dtypes: float64(2), int64(1), object(6)
memory usage: 702.8+ KB

In [0]:
df.head()

Out[8]:

,Order ID,Ship Mode,Segment,Region,Product ID,Sales,Quantity,Discount,Profit
0,CA-2016-152156,Second Class,Consumer,South,FUR-BO-10001798,261.9600,2,0%,41.9136
1,CA-2016-152156,Second Class,Consumer,South,FUR-CH-10000454,731.9400,3,0%,219.5820
2,CA-2016-138688,Second Class,Corporate,West,OFF-LA-10000240,14.6200,2,0%,6.8714
3,US-2015-108966,Standard Class,Consumer,South,FUR-TA-10000577,957.5775,5,0.45%,-383.0310
4,US-2015-108966,Standard Class,Consumer,South,OFF-ST-10000760,22.3680,2,0.20%,2.5164


In [0]:
df['Discount'] = df['Discount'].str.rstrip("%").astype(float)/100

In [0]:
df.head()

Out[10]:

,Order ID,Ship Mode,Segment,Region,Product ID,Sales,Quantity,Discount,Profit
0,CA-2016-152156,Second Class,Consumer,South,FUR-BO-10001798,261.9600,2,0.0000,41.9136
1,CA-2016-152156,Second Class,Consumer,South,FUR-CH-10000454,731.9400,3,0.0000,219.5820
2,CA-2016-138688,Second Class,Corporate,West,OFF-LA-10000240,14.6200,2,0.0000,6.8714
3,US-2015-108966,Standard Class,Consumer,South,FUR-TA-10000577,957.5775,5,0.0045,-383.0310
4,US-2015-108966,Standard Class,Consumer,South,OFF-ST-10000760,22.3680,2,0.0020,2.5164


In [0]:
df['Order ID'] = df['Order ID'].astype('string')
df.head()

Out[11]:

,Order ID,Ship Mode,Segment,Region,Product ID,Sales,Quantity,Discount,Profit
0,CA-2016-152156,Second Class,Consumer,South,FUR-BO-10001798,261.9600,2,0.0000,41.9136
1,CA-2016-152156,Second Class,Consumer,South,FUR-CH-10000454,731.9400,3,0.0000,219.5820
2,CA-2016-138688,Second Class,Corporate,West,OFF-LA-10000240,14.6200,2,0.0000,6.8714
3,US-2015-108966,Standard Class,Consumer,South,FUR-TA-10000577,957.5775,5,0.0045,-383.0310
4,US-2015-108966,Standard Class,Consumer,South,OFF-ST-10000760,22.3680,2,0.0020,2.5164


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 9 columns):
 # Column Non-Null Count Dtype 
--- ------ -------------- ----- 
 0 Order ID 9994 non-null string 
 1 Ship Mode 9994 non-null object 
 2 Segment 9994 non-null object 
 3 Region 9994 non-null object 
 4 Product ID 9994 non-null object 
 5 Sales 9994 non-null float64
 6 Quantity 9994 non-null int64 
 7 Discount 9994 non-null float64
 8 Profit 9994 non-null float64
dtypes: float64(3), int64(1), object(4), string(1)
memory usage: 702.8+ KB

In [0]:
df['Ship Mode'] = df['Ship Mode'].astype('string')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 9 columns):
 # Column Non-Null Count Dtype 
--- ------ -------------- ----- 
 0 Order ID 9994 non-null string 
 1 Ship Mode 9994 non-null string 
 2 Segment 9994 non-null object 
 3 Region 9994 non-null object 
 4 Product ID 9994 non-null object 
 5 Sales 9994 non-null float64
 6 Quantity 9994 non-null int64 
 7 Discount 9994 non-null float64
 8 Profit 9994 non-null float64
dtypes: float64(3), int64(1), object(3), string(2)
memory usage: 702.8+ KB

In [0]:
df['Segment'] = df['Segment'].astype('string')
df.head()

Out[14]:

,Order ID,Ship Mode,Segment,Region,Product ID,Sales,Quantity,Discount,Profit
0,CA-2016-152156,Second Class,Consumer,South,FUR-BO-10001798,261.9600,2,0.0000,41.9136
1,CA-2016-152156,Second Class,Consumer,South,FUR-CH-10000454,731.9400,3,0.0000,219.5820
2,CA-2016-138688,Second Class,Corporate,West,OFF-LA-10000240,14.6200,2,0.0000,6.8714
3,US-2015-108966,Standard Class,Consumer,South,FUR-TA-10000577,957.5775,5,0.0045,-383.0310
4,US-2015-108966,Standard Class,Consumer,South,OFF-ST-10000760,22.3680,2,0.0020,2.5164


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 9 columns):
 # Column Non-Null Count Dtype 
--- ------ -------------- ----- 
 0 Order ID 9994 non-null string 
 1 Ship Mode 9994 non-null string 
 2 Segment 9994 non-null string 
 3 Region 9994 non-null object 
 4 Product ID 9994 non-null object 
 5 Sales 9994 non-null float64
 6 Quantity 9994 non-null int64 
 7 Discount 9994 non-null float64
 8 Profit 9994 non-null float64
dtypes: float64(3), int64(1), object(2), string(3)
memory usage: 702.8+ KB

In [0]:
df['Region'] = df['Region'].astype('string')
df.head()

Out[16]:

,Order ID,Ship Mode,Segment,Region,Product ID,Sales,Quantity,Discount,Profit
0,CA-2016-152156,Second Class,Consumer,South,FUR-BO-10001798,261.9600,2,0.0000,41.9136
1,CA-2016-152156,Second Class,Consumer,South,FUR-CH-10000454,731.9400,3,0.0000,219.5820
2,CA-2016-138688,Second Class,Corporate,West,OFF-LA-10000240,14.6200,2,0.0000,6.8714
3,US-2015-108966,Standard Class,Consumer,South,FUR-TA-10000577,957.5775,5,0.0045,-383.0310
4,US-2015-108966,Standard Class,Consumer,South,OFF-ST-10000760,22.3680,2,0.0020,2.5164


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 9 columns):
 # Column Non-Null Count Dtype 
--- ------ -------------- ----- 
 0 Order ID 9994 non-null string 
 1 Ship Mode 9994 non-null string 
 2 Segment 9994 non-null string 
 3 Region 9994 non-null string 
 4 Product ID 9994 non-null object 
 5 Sales 9994 non-null float64
 6 Quantity 9994 non-null int64 
 7 Discount 9994 non-null float64
 8 Profit 9994 non-null float64
dtypes: float64(3), int64(1), object(1), string(4)
memory usage: 702.8+ KB

In [0]:
df['Product ID'] = df['Product ID'].astype('string')
df.head()

Out[18]:

,Order ID,Ship Mode,Segment,Region,Product ID,Sales,Quantity,Discount,Profit
0,CA-2016-152156,Second Class,Consumer,South,FUR-BO-10001798,261.9600,2,0.0000,41.9136
1,CA-2016-152156,Second Class,Consumer,South,FUR-CH-10000454,731.9400,3,0.0000,219.5820
2,CA-2016-138688,Second Class,Corporate,West,OFF-LA-10000240,14.6200,2,0.0000,6.8714
3,US-2015-108966,Standard Class,Consumer,South,FUR-TA-10000577,957.5775,5,0.0045,-383.0310
4,US-2015-108966,Standard Class,Consumer,South,OFF-ST-10000760,22.3680,2,0.0020,2.5164


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 9 columns):
 # Column Non-Null Count Dtype 
--- ------ -------------- ----- 
 0 Order ID 9994 non-null string 
 1 Ship Mode 9994 non-null string 
 2 Segment 9994 non-null string 
 3 Region 9994 non-null string 
 4 Product ID 9994 non-null string 
 5 Sales 9994 non-null float64
 6 Quantity 9994 non-null int64 
 7 Discount 9994 non-null float64
 8 Profit 9994 non-null float64
dtypes: float64(3), int64(1), string(5)
memory usage: 702.8 KB

In [0]:
df.drop_duplicates()

Out[20]:

,Order ID,Ship Mode,Segment,Region,Product ID,Sales,Quantity,Discount,Profit
0,CA-2016-152156,Second Class,Consumer,South,FUR-BO-10001798,261.9600,2,0.0000,41.9136
1,CA-2016-152156,Second Class,Consumer,South,FUR-CH-10000454,731.9400,3,0.0000,219.5820
2,CA-2016-138688,Second Class,Corporate,West,OFF-LA-10000240,14.6200,2,0.0000,6.8714
3,US-2015-108966,Standard Class,Consumer,South,FUR-TA-10000577,957.5775,5,0.0045,-383.0310
4,US-2015-108966,Standard Class,Consumer,South,OFF-ST-10000760,22.3680,2,0.0020,2.5164
...,...,...,...,...,...,...,...,...,...
9989,CA-2014-110422,Second Class,Consumer,South,FUR-FU-10001889,25.2480,3,0.0020,4.1028
9990,CA-2017-121258,Standard Class,Consumer,West,FUR-FU-10000747,91.9600,2,0.0000,15.6332
9991,CA-2017-121258,Standard Class,Consumer,West,TEC-PH-10003645,258.5760,2,0.0020,19.3932
9992,CA-2017-121258,Standard Class,Consumer,West,OFF-PA-10004041,29.6000,4,0.0000,13.3200


In [0]:
dbutils.fs.unmount('/mnt/lake1')

/mnt/lake1 has been unmounted.
Out[21]: True

In [0]:
dbutils.fs.mount(
  source = "wasbs://silver@projectaccount123.blob.core.windows.net",
  mount_point = '/mnt/lake1',
  extra_configs = {
    'fs.azure.account.key.projectaccount123.blob.core.windows.net':'c9F/bns3QWX59Hx7IDic1V9iSaoB17Gk8y5f0npreZMROsyQ6ausbUeBBUmpGjdtKq17pnBwSOMq+AStiJsEnQ=='
  }
  
)

Out[22]: True

In [0]:
df = spark.createDataFrame(df)

/databricks/spark/python/pyspark/sql/pandas/conversion.py:340: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
 Cannot specify a mask or a size when passing an object that is converted with the __arrow_array__ protocol.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
 warnings.warn(msg)

In [0]:
df.coalesce(1).write.format("com.databricks.spark.csv").option("header",'true').mode("overwrite").save("/mnt/lake1/testoutput.csv")

In [0]:
dbutils.fs.ls('/mnt/lake1')

Out[31]: [FileInfo(path='dbfs:/mnt/lake1/testoutput.csv/', name='testoutput.csv/', size=0, modificationTime=0)]

##2 - Spark Configs

With Spark configs, the Azure Data Lake Store settings can be specified per notebook. To keep things simple, the example below includes the credentials in plaintext. However, we strongly discourage you from storing secrets in plaintext. Instead, we recommend storing the credentials as [Databricks Secrets](https://docs.azuredatabricks.net/user-guide/secrets/index.html#secrets-user-guide).

**Note:** `spark.conf` values are visible only to the DataSet and DataFrames API. If you need access to them from an RDD, refer to the [documentation](https://docs.azuredatabricks.net/spark/latest/data-sources/azure/azure-datalake.html#access-azure-data-lake-store-using-the-rdd-api).

In [0]:
%scala
spark.conf.set("dfs.adls.oauth2.access.token.provider.type", "ClientCredential")
spark.conf.set("dfs.adls.oauth2.client.id", "<aaaaaaaa-bbbb-cccc-dddd-eeeeeeeeeeee>")
spark.conf.set("dfs.adls.oauth2.credential", "<NzQzY2QzYTAtM2I3Zi00NzFmLWI3MGMtMzc4MzRjZmk=>")
spark.conf.set("dfs.adls.oauth2.refresh.url", "https://login.microsoftonline.com/<ffffffff-gggg-hhhh-iiii-jjjjjjjjjjjj>/oauth2/token")

In [0]:
%fs ls adl://kpadls.azuredatalakestore.net/testing/

In [0]:
%scala
spark.read.parquet("dbfs:/mnt/my-datasets/datasets/iot/events").write.mode("overwrite").parquet("adl://kpadls.azuredatalakestore.net/testing/tmp/kp/v1")

In [0]:
%fs ls adl://kpadls.azuredatalakestore.net/testing/tmp/kp/v1